### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns

In [2]:
pd.options.display.max_columns= None
pd.options.display.max_colwidth= None
pd.options.display.max_rows = None

## Read dataset

In [3]:
df = pd.read_excel('/home/mahdi/NLP/Text_Preprocessing/dataset/Comment_Full_Data_Sentiment_Labeled.xlsx')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131 entries, 0 to 1130
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   comment_id              1131 non-null   int64  
 1   app_id                  1131 non-null   int64  
 2   user_name               1131 non-null   object 
 3   comment_text            1131 non-null   object 
 4   Annotator1              1130 non-null   float64
 5   Annotator2              1129 non-null   float64
 6   Annotator3              1131 non-null   int64  
 7   comment_rating          1131 non-null   int64  
 8   comment_date            1131 non-null   int64  
 9   sentiment_result        1131 non-null   object 
 10  second_model_processed  1131 non-null   bool   
 11  comment_idd             1131 non-null   int64  
 12  sentiment_score         1131 non-null   int64  
 13  comment_date_jalali     1131 non-null   int64  
dtypes: bool(1), float64(2), int64(8), object

In [5]:
df.head()

,comment_id,app_id,user_name,comment_text,Annotator1,Annotator2,Annotator3,comment_rating,comment_date,sentiment_result,second_model_processed,comment_idd,sentiment_score,comment_date_jalali
0,111177,9,هادی,سلام،خوبه ولی برنامه همراه شهر باز نمیشه.,3.0,3.0,3,5,45650,mixed,False,175629681,0,14031004
1,111172,9,Unnamed User,سلام خسته نباشید از این برنامه میشه غیر حضوری حساب باز کنیم یا نه ؟,0.0,0.0,0,5,45649,no sentiment expressed,False,175659112,0,14031003
2,110746,8,Unnamed User,لطفا تعداد اقساط رو از ۱۲ ماه شروع کنید نه ۲۴ ماه,0.0,0.0,0,5,45651,no sentiment expressed,False,175712781,0,14031005
3,110706,8,shayan,خود بانک و وام هاش خوبه ولی نام افزارش خیلی هنگی و قطعی داره,3.0,3.0,3,2,45651,mixed,False,175740844,0,14031005
4,110702,8,hamed,سلام چرا وارد همراه بانک نمیتونم بشم,2.0,2.0,2,3,45651,no sentiment expressed,False,175741764,0,14031005


## Preprocessing

In [6]:
def _multiple_replace(mapping, text):
    pattern = "|".join(map(re.escape, mapping.keys()))
    return re.sub(pattern, lambda m: mapping[m.group()], str(text))

def convert_fa_numbers(input_str):
    mapping = {
        '۰': '0',
        '۱': '1',
        '۲': '2',
        '۳': '3',
        '۴': '4',
        '۵': '5',
        '۶': '6',
        '۷': '7',
        '۸': '8',
        '۹': '9',
        '.': '.',
    }
    return _multiple_replace(mapping, input_str)

def convert_en_numbers(input_str):
    mapping = {
         '0': '۰',
         '1' : '۱',
         '2' :'۲',
        '3'  :'۳',
        '4'  :'۴',
        '5' :'۵',
        '6' :'۶',
        '7' :'۷',
        '8' :'۸',
        '9' :'۹',
        '.' :'.'
    }
    return _multiple_replace(mapping, input_str)

def convert_ar_characters(input_str):
    """
    Converts Arabic chars to related Persian unicode char
    """
    mapping = {
        'ك': 'ک',
        'ى': 'ی',
        'ي': 'ی',
        'ئ':'ی',
        'إ':'ا',
        'أ':'ا',
        'ة':'ه',
        'ؤ':'و'
    }
    return _multiple_replace(mapping, input_str)



### Convert Emojis

In [145]:
import emoji
import re

def convert_emojis_to_persian(text):
    # First, convert emojis to English descriptive labels.
    # This will convert "😊" into something like ":smiling_face_with_smiling_eyes:".
    demojized_text = emoji.demojize(text)
    
    # Define a dictionary mapping some common English emoji labels to Persian words.
    persian_emoji_map = {
        ":smiling_face_with_smiling_eyes:": "خندان",
        ":grinning_face:": "با لبخند",
        ":face_with_tears_of_joy:": "با اشک شوق",
        ":red_heart:": "عشق",
        ":thumbs_up:": "پسندیده",
        ":thumbs_down:" : "نپسندیده" , # 👎
        ":OK_hand:" : "خوب", # 👌
        "folded_hands": "تشکر"  ,
        "rose" : "مرسی" , 
        "cherry_blossom" : "سپاس" ,
        "face_with_symbols_on_mouth" : "خیلی عصبانی" , 
        ":face_vomiting:" : "مزخرف" , #  🤮
        ":angry_face:" : "عصبانی", # 😠
        ":broken_heart:" : "قلب شکسته", # 💔
        ":clapping_hands:" : "عالی" , # 👏 
        ":confused_face:" : "گیج شدم" , # 😕 
        ":crying_face:" : "گریه میکنم" , # 😢 
        ":disappointed_face:" : "ناامید", # 😞 
        ":enraged_face:" : "عصبانی" , # 😡
        ":expressionless_face:" : "خنثی" , # 😑
        ":sparkling_heart:" : "دوستداشتنی" , # 💖
        ":smiling_face_with_heart-eyes:" : "دوستداشتنی", # 😍
    
    
    }
# ":crossed_fingers:" : "" , # 🤞   
# ":downcast_face_with_sweat:" : "" , # 😓    
# 😘 → :face_blowing_a_kiss:
# 😮‍💨 → :face_exhaling:
# 🥹 → :face_holding_back_tears:
# 😋 → :face_savoring_food:
# 😱 → :face_screaming_in_fear:
# 😵 → :face_with_crossed-out_eyes:
# 🫤 → :face_with_diagonal_mouth:
# 🤭 → :face_with_hand_over_mouth:
# 🤕 → :face_with_head-bandage:
# 😷 → :face_with_medical_mask:
# 🧐 → :face_with_monocle:
# 🫢 → :face_with_open_eyes_and_hand_over_mouth:
# 😮 → :face_with_open_mouth:
# 🫣 → :face_with_peeking_eye:
# 🤨 → :face_with_raised_eyebrow:
# 🙄 → :face_with_rolling_eyes:
# 😵‍💫 → :face_with_spiral_eyes:
# 😤 → :face_with_steam_from_nose:
# 😂 → :face_with_tears_of_joy:
# 🤒 → :face_with_thermometer:
# 😛 → :face_with_tongue:
# 😶 → :face_without_mouth:
# 🔥 → :fire:
# 💪 → :flexed_biceps:
# 😳 → :flushed_face:
# 😗 → :kissing_face:
# 😚 → :kissing_face_with_closed_eyes:
# 😙 → :kissing_face_with_smiling_eyes:
# 😭 → :loudly_crying_face:
# 😐 → :neutral_face:
# 🥳 → :partying_face:
# 🙁 → :slightly_frowning_face:
# 🙂 → :slightly_smiling_face:
# 😇 → :smiling_face_with_halo:

# 🥰 → :smiling_face_with_hearts:
# 😈 → :smiling_face_with_horns:
# 🤗 → :smiling_face_with_open_hands:
# 😊 → :smiling_face_with_smiling_eyes:
# 😎 → :smiling_face_with_sunglasses:
# 🥲 → :smiling_face_with_tear:
# 😏 → :smirking_face:
# 🤩 → :star-struck:
# 🤔 → :thinking_face:
# 😒 → :unamused_face:
# ✌🏻 → :victory_hand_light_skin_tone:
# 👋 → :waving_hand:
# 😜 → :winking_face_with_tongue:
# 🥴 → :woozy_face:
# 😟 → :worried_face:






    # Replace known emojis with Persian equivalents
    for eng_label, pers_label in persian_emoji_map.items():
        persian_formatted = f"[{pers_label}]"  # Wrap in brackets
        demojized_text = re.sub(re.escape(eng_label), persian_formatted, demojized_text)

    # Replace any remaining :emoji_name: patterns (unmapped emojis) with a space
    demojized_text = re.sub(r':[a-zA-Z0-9_]+:', ' ', demojized_text)
    
    return demojized_text



In [107]:
# Get all emojis and their descriptions
for emo, data in emoji.EMOJI_DATA.items():
    print(f"{emo} → {data['en']}")

🥇 → :1st_place_medal:
🥈 → :2nd_place_medal:
🥉 → :3rd_place_medal:
🆎 → :AB_button_(blood_type):
🏧 → :ATM_sign:
🅰️ → :A_button_(blood_type):
🅰 → :A_button_(blood_type):
🇦🇫 → :Afghanistan:
🇦🇱 → :Albania:
🇩🇿 → :Algeria:
🇦🇸 → :American_Samoa:
🇦🇩 → :Andorra:
🇦🇴 → :Angola:
🇦🇮 → :Anguilla:
🇦🇶 → :Antarctica:
🇦🇬 → :Antigua_&_Barbuda:
♒ → :Aquarius:
🇦🇷 → :Argentina:
♈ → :Aries:
🇦🇲 → :Armenia:
🇦🇼 → :Aruba:
🇦🇨 → :Ascension_Island:
🇦🇺 → :Australia:
🇦🇹 → :Austria:
🇦🇿 → :Azerbaijan:
🔙 → :BACK_arrow:
🅱️ → :B_button_(blood_type):
🅱 → :B_button_(blood_type):
🇧🇸 → :Bahamas:
🇧🇭 → :Bahrain:
🇧🇩 → :Bangladesh:
🇧🇧 → :Barbados:
🇧🇾 → :Belarus:
🇧🇪 → :Belgium:
🇧🇿 → :Belize:
🇧🇯 → :Benin:
🇧🇲 → :Bermuda:
🇧🇹 → :Bhutan:
🇧🇴 → :Bolivia:
🇧🇦 → :Bosnia_&_Herzegovina:
🇧🇼 → :Botswana:
🇧🇻 → :Bouvet_Island:
🇧🇷 → :Brazil:
🇮🇴 → :British_Indian_Ocean_Territory:
🇻🇬 → :British_Virgin_Islands:
🇧🇳 → :Brunei:
🇧🇬 → :Bulgaria:
🇧🇫 → :Burkina_Faso:
🇧🇮 → :Burundi:
🆑 → :CL_button:
🆒 → :COOL_button:
🇰🇭 → :Cambodia:
🇨🇲 → :Cameroon:
🇨🇦 → :Canad

## preprocess functions

In [83]:

def merge_mi_prefix(text):

    return re.sub(r'\b(ن?می)\s+(\S+)', r'\1\2', text)

def remove_diacritics(text):
    # Define regex for Persian diacritics (Unicode range: \u064B-\u0652)
    diacritics_pattern = re.compile(r'[\u064B-\u0652]')
    return re.sub(diacritics_pattern, '', text)


def convert_number_to_text(text):
    
    return re.sub(r'(\d)\d*', r'\1', text)



def map_num_to_text(text):

    # Check if the text is exactly '1', '2', or '3' (nothing else)
    mapping = {'1': 'خیلی بد', '2': 'بد', '3': 'متوسط' , '4': 'خوب' , '5': 'عالی'}
    
    if text in mapping:
        return mapping[text]  # Replace number with label
    
    return text 


## stemming and lemmatization

#### Using hazm library

In [ ]:
from hazm import word_tokenize, Normalizer
normalizer = Normalizer()
normalized_text = normalizer.normalize(text)
tokens = word_tokenize(normalized_text)


In [ ]:
from hazm import Stemmer
stemmer = Stemmer()
stemmed_tokens = [stemmer.stem(token) for token in tokens]


### using parsivar library

### Using Dadma library

## Main Function

In [139]:
def preprocess(text,
               convert_farsi_numbers = False,
               convert_english_numbers = False,
               convert_arabic_characters = False,
               remove_diacritic = False,
               convert_emojis = False,
               remove_half_space = False,
               remove_removelist = False,
               remove_extra_characters = False,
               remove_numbers = False,
               remove_punctuation = False,
               replace_multiple_space = False,
               handle_prefix = False,
               map_number_to_text = False
               
               ):
    
    text = text.strip()

    if convert_farsi_numbers:
        text = convert_fa_numbers(text)

    if convert_arabic_characters:
# convert arabic characters to persian
        text = convert_ar_characters(text)

    if remove_diacritic:
        text = remove_diacritics(text)

    if convert_emojis:
# convert Emojis
        text = convert_emojis_to_persian(text)
    
    if remove_removelist:
        removelist = "<>"
        # text = re.sub(r'[^\w'+removelist+']', ' ', text)
        # text = re.sub(r'[^\w]', ' ', text)
        # text = re.sub(r'((#)[\w]*)','#',text)
    
    if remove_half_space:
# remove half space
        text = text.replace('\u200c', '')
    
    if remove_extra_characters:
        text = re.sub(r'(\w)\1{2,}', r'\1\1',text)

    if map_number_to_text:
        text = map_num_to_text(text)

# remove numbers
    if remove_numbers:
        text = re.sub(r' [\d+]', ' ',text)

# convert english numbers to persian
    if convert_english_numbers:
        text = convert_en_numbers(text)
    
# remove punctuations
    if remove_punctuation:
        text= re.sub(r'[^\w\[\]]', ' ', text)

# replace multiple spaces with one space
    if replace_multiple_space:
        text = re.sub(r'[\s]{2,}', ' ', text)

# prefix
    if handle_prefix:
        text = merge_mi_prefix(text)
   
    
    return(text)

## Test the function

In [105]:
text = 'سلام. این شماره ۵ مال کدوم شماره از ۱۲۳۴۵ هست'
convert_number_to_text(text)

In [87]:
# Example inputs:
# print(map_num_to_text("1"))        # Output: خیلی بد
print(map_num_to_text(" 12 "))      # Output: بد
# print(map_number_to_text("3"))        # Output: متوسط
# print(map_number_to_text("سلام 1"))   # Output: سلام 1
# print(map_number_to_text("۱۲۳"))      # Output: ۱۲۳
# print(map_number_to_text(" شماره 3 ")) # Output:  شماره 3 

 12 


In [110]:
# text = df[df["comment_id"]== 93575]['comment_text']
text = df[df["comment_id"]== 94729]['comment_text']
text

1009    در هنگام ثبت نام کد دعوت yg6iamnz بزنید تا ۷۰ هزارتومان پاداش براتون واریز شه با تشکر ✅️✅️yg6iamnz✅️✅️
Name: comment_text, dtype: object

In [111]:
text.apply(lambda x : map_num_to_text(x))

1009    در هنگام ثبت نام کد دعوت yg6iamnz بزنید تا ۷۰ هزارتومان پاداش براتون واریز شه با تشکر ✅️✅️yg6iamnz✅️✅️
Name: comment_text, dtype: object

In [103]:
text = ' سلامممم.!!! من نمی خواهم ازین جا ببببرم.... نمره ۵ می دم!؟.'

In [47]:
preprocess(text)

'سلامم من نمیخواهم ازین جا ببرم نمره 5 میدم '

In [48]:
# Example Usage
text_with_diacritics = "مُسلم، مَدرسه، اَمسْ"
clean_text = remove_diacritics(text_with_diacritics)
print(clean_text)  

مسلم، مدرسه، امس


In [122]:
text = df[df["comment_id"]== 99713]['comment_text']
# text = df[df["comment_id"]== 110441]['comment_text']
# text = '👍👍👍👍'
text

898    🤬
Name: comment_text, dtype: object

In [117]:
# convert_emojis_to_persian(text)

In [126]:
# preprocess(text)

In [123]:
text.apply(lambda x: preprocess(text = x,
           convert_farsi_numbers = True,
               convert_english_numbers = False,
               convert_arabic_characters = True,
               remove_diacritic = True,
               convert_emojis = True,
               remove_half_space = False,
               remove_removelist = False,
               remove_extra_characters = True,
               remove_numbers = False,
               remove_punctuation = True,
               replace_multiple_space = True,
               handle_prefix = True,
               map_number_to_text = True
               
           ))

898     face_with_symbols_on_mouth 
Name: comment_text, dtype: object

In [106]:
preprocess(text = text,
           convert_farsi_numbers = True,
               convert_english_numbers = False,
               convert_arabic_characters = True,
               remove_diacritic = True,
               convert_emojis = True,
               remove_half_space = False,
               remove_removelist = False,
               remove_extra_characters = True,
               remove_numbers = False,
               remove_punctuation = True,
               replace_multiple_space = True,
               handle_prefix = True,
               map_number_to_text = True
               
           )

'سلام این شماره 5 مال کدوم شماره از 12345 هست'

## Total Test

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131 entries, 0 to 1130
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   comment_id              1131 non-null   int64  
 1   app_id                  1131 non-null   int64  
 2   user_name               1131 non-null   object 
 3   comment_text            1131 non-null   object 
 4   Annotator1              1130 non-null   float64
 5   Annotator2              1129 non-null   float64
 6   Annotator3              1131 non-null   int64  
 7   comment_rating          1131 non-null   int64  
 8   comment_date            1131 non-null   int64  
 9   sentiment_result        1131 non-null   object 
 10  second_model_processed  1131 non-null   bool   
 11  comment_idd             1131 non-null   int64  
 12  sentiment_score         1131 non-null   int64  
 13  comment_date_jalali     1131 non-null   int64  
dtypes: bool(1), float64(2), int64(8), object

In [141]:
df_reduced = df.drop(['app_id','user_name','Annotator1','Annotator2','comment_rating','second_model_processed','comment_idd','sentiment_score','comment_date'] , axis=1)

In [142]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131 entries, 0 to 1130
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   comment_id           1131 non-null   int64 
 1   comment_text         1131 non-null   object
 2   Annotator3           1131 non-null   int64 
 3   sentiment_result     1131 non-null   object
 4   comment_date_jalali  1131 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 44.3+ KB


In [146]:
df_reduced['comment_preprocessed'] = df_reduced['comment_text'].apply(lambda x: preprocess(text = x,
           convert_farsi_numbers = True,
               convert_english_numbers = False,
               convert_arabic_characters = True,
               remove_diacritic = True,
               convert_emojis = True,
               remove_half_space = False,
               remove_removelist = False,
               remove_extra_characters = True,
               remove_numbers = False,
               remove_punctuation = True,
               replace_multiple_space = True,
               handle_prefix = True,
               map_number_to_text = True
               
           ))

In [147]:
df_reduced.head(100)

,comment_id,comment_text,Annotator3,sentiment_result,comment_date_jalali,comment_preprocessed
0,111177,سلام،خوبه ولی برنامه همراه شهر باز نمیشه.,3,mixed,14031004,سلام خوبه ولی برنامه همراه شهر باز نمیشه
1,111172,سلام خسته نباشید از این برنامه میشه غیر حضوری حساب باز کنیم یا نه ؟,0,no sentiment expressed,14031003,سلام خسته نباشید از این برنامه میشه غیر حضوری حساب باز کنیم یا نه
2,110746,لطفا تعداد اقساط رو از ۱۲ ماه شروع کنید نه ۲۴ ماه,0,no sentiment expressed,14031005,لطفا تعداد اقساط رو از 12 ماه شروع کنید نه 24 ماه
3,110706,خود بانک و وام هاش خوبه ولی نام افزارش خیلی هنگی و قطعی داره,3,mixed,14031005,خود بانک و وام هاش خوبه ولی نام افزارش خیلی هنگی و قطعی داره
4,110702,سلام چرا وارد همراه بانک نمیتونم بشم,2,no sentiment expressed,14031005,سلام چرا وارد همراه بانک نمیتونم بشم
5,110668,عالی فقط بعضی وقتا دیر نشون میده ک چقدر ب حسابت پول اومده,4,positive,14031007,عالی فقط بعضی وقتا دیر نشون میده ک چقدر ب حسابت پول اومده
6,110667,بهترین بانک عالی ی چند روزی هست سیستم هاش در حال بروز رسانیه ی کم اختلال داره درست میشه واقعا بانک خوبیه,4,very positive,14031007,بهترین بانک عالی ی چند روزی هست سیستم هاش در حال بروز رسانیه ی کم اختلال داره درست میشه واقعا بانک خوبیه
7,110624,خوب بود کلی باش کار کردم ولی امروز باز نميشه حال میخوام پول انتقال بدم نميشه,3,no sentiment expressed,14031005,خوب بود کلی باش کار کردم ولی امروز باز نمیشه حال میخوام پول انتقال بدم نمیشه
8,110618,چند تا از نظرها را خوندم دیدم بی‌انصافیه واقعیتو نو نظرم نگم کاری به قضاوت ندارم هیچ سودیم برا من نداره ولی در کل می‌خوام بگم این بانک بانکیه که مردمو درک می‌کنه کارمزدی هم که می‌گیره از همه بانک‌ها کمتره و مهمتر به روز در کل خاص‌ترین و بهترین بانک مردم شناس,5,very positive,14031005,چند تا از نظرها را خوندم دیدم بی انصافیه واقعیتو نو نظرم نگم کاری به قضاوت ندارم هیچ سودیم برا من نداره ولی در کل میخوام بگم این بانک بانکیه که مردمو درک میکنه کارمزدی هم که میگیره از همه بانک ها کمتره و مهمتر به روز در کل خاص ترین و بهترین بانک مردم شناس
9,110525,با سلام خیلی عالیه چون مبالغ بالا رو میتونی با همراه بانک انجام بدی,5,very positive,14031007,با سلام خیلی عالیه چون مبالغ بالا رو میتونی با همراه بانک انجام بدی
